In [1]:
import numpy as np
import pandas as pd
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm

In [2]:
HOME_DIR = '/data/amandeep/nih-dataset/organization'
table_path = f'{HOME_DIR}/org_for_tl_with_qnode.tsv'
f_name = table_path.split("/")[-1]
wikify_column_name = "name,city,state,country"
final_score_column = "siamese_prediction"

canonical_file_path = f'{HOME_DIR}/temp/canonical.csv'
candidate_file_path = f'{HOME_DIR}/temp/candidates.csv'
aux_field = 'graph_embedding_complex,class_count,property_count,context'
temp_dir= f'{HOME_DIR}/temp/temp'

aligned_pagerank_candidate_file_path = f'{HOME_DIR}/temp/apr_test.csv'
model_file_path = './models/weighted_lr.pkl'
ranking_model_file_path = './models/epoch_5_loss_0.09882864356040955_top1_0.8968926553672316.pth'
min_max_scaler_path = './models/normalization_factor.pkl'

model_voted_candidate_file_path = f'{HOME_DIR}/temp/mv_test.csv'
graph_embedding_file_path = f'{HOME_DIR}/temp/score_test.csv'

lof_reciprocal_rank_file_path = f'{HOME_DIR}/temp/lof_rr_test.csv'
lof_tfidf_file_path = f'{HOME_DIR}/temp/lof_tfidf_test.csv'
lof_feature_file = f'{HOME_DIR}/temp/lof_feature.csv'
context_score_file = f'{HOME_DIR}/temp/context_score_file.csv'

output_model_pred_file = f'{HOME_DIR}/temp/model_prediction.csv'
top5_links = f'{HOME_DIR}/temp/top5_links.csv'
colorized_kg_links = f'{HOME_DIR}/temp/{f_name.strip(".csv")}_colorized.xlsx'

graph_embedding_complex_file = f'{HOME_DIR}/temp/graph_embedding_complex.tsv'
class_count_file = f'{HOME_DIR}/temp/class_count.tsv'
property_count_file = f'{HOME_DIR}/temp/property_count.tsv'
context_file = f'{HOME_DIR}/temp/context.tsv'
index_url = 'http://ckg07:9200/wikidatadwd-augmented/'

string_threshold = 0.9
siamese_threshold = 0.9
custom_context_file = '/Users/grantxie/test/coi3/coauthors.context.tsv.gz'
gt = '/Users/grantxie/Downloads/groundtruth_new.csv'
selection_save_path = 'test_selection.csv'
labeled_path = 'test_eva.csv'

In [3]:
features = ['pagerank','retrieval_score','monge_elkan','monge_elkan_aliases','des_cont_jaccard',
            'jaro_winkler','levenshtein','singleton','num_char','num_tokens',
           'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
           'lof-graph-embedding-score', 'lof-reciprocal-rank', 'context_score']

In [4]:
print(len(features))

15


In [5]:
!ls "$table_path"

/data/amandeep/nih-dataset/organization/org_for_tl_with_qnode.tsv


### Canonicalize

In [6]:
!tl canonicalize -c "$wikify_column_name" --add-context "$table_path" -s org_node,city_node,state_node,country_node --tsv \
> "$canonical_file_path"

canonicalize Time: 0.13051271438598633s


In [7]:
pd.read_csv(canonical_file_path, nrows = 5)

,column,row,label,context,filename,column-id
0,0,0,UNIVERSITY OF WASHINGTON,SEATTLE|WA|UNITED STATES,org_for_tl_with_qnode.tsv,org_for_tl_with_qnode.tsv-0
1,0,1,CHARLES R. DREW UNIVERSITY OF MEDICAL & SCIENCE,LOS ANGELES|CA|UNITED STATES,org_for_tl_with_qnode.tsv,org_for_tl_with_qnode.tsv-0
2,0,2,"UNIVERSITY OF CALIFORNIA, SAN DIEGO",LA JOLLA|CA|UNITED STATES,org_for_tl_with_qnode.tsv,org_for_tl_with_qnode.tsv-0
3,0,3,UNIVERSITY OF MIAMI SCHOOL OF MEDICINE,CORAL GABLES|FL|UNITED STATES,org_for_tl_with_qnode.tsv,org_for_tl_with_qnode.tsv-0
4,0,4,BAYLOR UNIVERSITY,WACO|TX|UNITED STATES,org_for_tl_with_qnode.tsv,org_for_tl_with_qnode.tsv-0


In [8]:
!ls -l $canonical_file_path

-rw-r--r-- 1 amandeep isdstaff 2742224 Aug  2 17:48 /data/amandeep/nih-dataset/organization/temp/canonical.csv


In [9]:
def split(f_path, output_path):
    df = pd.read_csv(f_path)
    for column, gdf in df.groupby(by=['column']):
        print(column, len(gdf))
        d_list = np.array_split(gdf, 10)
        for i, d in enumerate(d_list):
            d.to_csv(f'{output_path}/split_{column}_{i}.csv', index=False)

In [10]:
split(canonical_file_path, '/data/amandeep/nih-dataset/organization/nih-split')

0 6025
1 6025
2 6025
3 6025


# Candidate Generation

In [11]:
def candidate_generation(path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
        print(f"{filename}: {i+1} of {len(file_list)}")
        output_file = f"{output_path}/{filename}"
        
        !tl clean -c label -o label_clean $file / \
        --url http://ckg07:9200 --index wikidatadwd-augmented \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url http://ckg07:9200 --index wikidatadwd-augmented \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" > "$output_file"
        
        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename[:-4]}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename[:-4]}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename[:-4]}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename[:-4]}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)

In [12]:
input_path = '/data/amandeep/nih-dataset/organization/nih-split'
output_path = '/data/amandeep/nih-dataset/organization/candidates'
!mkdir -p $output_path

class_count_path = '/data/amandeep/nih-dataset/organization/temp/class_c'
!mkdir -p $class_count_path
prop_count_path = '/data/amandeep/nih-dataset/organization/temp/prop_c'
!mkdir -p $prop_count_path
context_path = '/data/amandeep/nih-dataset/organization/temp/context'
!mkdir -p $context_path
graph_embedding = '/data/amandeep/nih-dataset/organization/temp/ge'
!mkdir -p $graph_embedding
import time

In [13]:
candidate_generation(input_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding)

0it [00:00, ?it/s]

split_0_0.csv: 1 of 40
clean Time: 0.03534221649169922s
get-fuzzy-augmented-matches Time: 44.592395544052124s
get-exact-matches Time: 3.4459550380706787s


1it [00:58, 58.83s/it]

split_0_1.csv: 2 of 40
clean Time: 0.03396940231323242s
get-fuzzy-augmented-matches Time: 43.86876440048218s
get-exact-matches Time: 3.6438634395599365s


2it [01:56, 58.31s/it]

...

38it [21:42, 16.52s/it]

split_3_8.csv: 39 of 40
clean Time: 0.02732062339782715s
get-fuzzy-augmented-matches Time: 8.29240608215332s
get-exact-matches Time: 1.5062415599822998s


39it [22:01, 17.15s/it]

split_3_9.csv: 40 of 40
clean Time: 0.026815176010131836s
get-fuzzy-augmented-matches Time: 7.332457542419434s
get-exact-matches Time: 1.5170671939849854s


40it [22:18, 33.47s/it]


In [14]:
features_path = '/data/amandeep/nih-dataset/organization/features'
!mkdir -p $features_path
classifier_features= ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']

In [15]:
def feature_generation(candidate_dir, embedding_dir, class_count_dir, property_count_dir, context_path, output_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        filename = file.split('/')[-1]
        print(f"{filename}: {i+1} of {len(file_list)}")
        embedding_file = f"{embedding_dir}/{filename[:-4]}_graph_embedding_complex.tsv"
        class_count_file = f"{class_count_dir}/{filename[:-4]}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename[:-4]}_prop_count.tsv"
        context_file = f"{context_path}/{filename[:-4]}_context.tsv"
        output_file = f"{output_path}/{filename}"
        if os.path.getsize(file) == 0:
                continue
        classifier_features_str = ",".join(classifier_features)
        !tl align-page-rank "$file" \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases \
            / string-similarity -i --method jaro_winkler -o jaro_winkler \
            / string-similarity -i --method levenshtein -o levenshtein \
            / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
            / normalize-scores -c des_cont_jaccard / smallest-qnode-number \
            / mosaic-features -c kg_labels --num-char --num-tokens \
            / create-singleton-feature -o singleton \
            / vote-by-classifier  \
            --prob-threshold 0.995 \
            --features "$classifier_features_str" \
            --model "$model_file_path" \
            / score-using-embedding \
            --column-vector-strategy centroid-of-lof \
            --lof-strategy ems-mv \
            -o lof-graph-embedding-score \
            --embedding-file "$embedding_file" \
            / generate-reciprocal-rank  \
            -c lof-graph-embedding-score \
            -o lof-reciprocal-rank \
            / compute-tf-idf  \
            --feature-file "$class_count_file" \
            --feature-name class_count \
            --singleton-column is_lof \
            -o lof_class_count_tf_idf_score \
            / compute-tf-idf \
            --feature-file "$property_count_file" \
            --feature-name property_count \
            --singleton-column is_lof \
            -o lof_property_count_tf_idf_score   > "$output_file"

In [16]:
feature_generation(output_path, graph_embedding, class_count_path, prop_count_path, context_path, features_path)

0it [00:00, ?it/s]

split_0_0.csv: 1 of 40
align-page-rank Time: 1.0750133991241455s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 34.22521376609802s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 60.546825885772705s
string-similarity-['jaro_winkler'] Time: 5.3626708984375s
string-similarity-['levenshtein'] Time: 42.36357617378235s
string-similarity-['jaccard:tokenizer=word'] Time: 0.8475470542907715s
normalize-scores-des_cont_jaccard Time: 0.24726581573486328s
smallest-qnode-number Time: 1.79902982711792s
mosaic-features Time: 0.11835169792175293s
create-singleton-feature Time: 1.5949225425720215s
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time: 1.3775908946990967s
Qnodes to lookup: 35492
Qnodes from file:

1it [03:07, 187.89s/it]

split_0_1.csv: 2 of 40
align-page-rank Time: 0.9559783935546875s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 27.57402729988098s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 53.09959959983826s
string-similarity-['jaro_winkler'] Time: 5.0380518436431885s
string-similarity-['levenshtein'] Time: 35.161452293395996s
string-similarity-['jaccard:tokenizer=word'] Time: 0.8215985298156738s
normalize-scores-des_cont_jaccard Time: 0.24749255180358887s
smallest-qnode-number Time: 1.7863819599151611s
mosaic-features Time: 0.11316394805908203s
create-singleton-feature Time: 1.3897294998168945s
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time: 0.7831606864929199s
Qnodes to lookup: 34635
Qnodes from 

2it [05:49, 172.31s/it]

...

38it [1:53:42, 370.33s/it]

split_3_8.csv: 39 of 40
align-page-rank Time: 1.0018589496612549s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 61.890140533447266s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 130.56145191192627s
string-similarity-['jaro_winkler'] Time: 11.904874324798584s
string-similarity-['levenshtein'] Time: 71.60157465934753s
string-similarity-['jaccard:tokenizer=word'] Time: 1.7614617347717285s
normalize-scores-des_cont_jaccard Time: 0.39958667755126953s
smallest-qnode-number Time: 2.393639087677002s
mosaic-features Time: 0.28822803497314453s
create-singleton-feature Time: 2.805938243865967s
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time: 1.7723073959350586s
Qnodes to lookup: 7701
Qnodes from f

39it [1:59:33, 364.80s/it]

split_3_9.csv: 40 of 40
align-page-rank Time: 1.0586178302764893s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 67.3774425983429s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 142.59416317939758s
string-similarity-['jaro_winkler'] Time: 12.160614013671875s
string-similarity-['levenshtein'] Time: 76.88030791282654s
string-similarity-['jaccard:tokenizer=word'] Time: 1.7703096866607666s
normalize-scores-des_cont_jaccard Time: 0.406919002532959s
smallest-qnode-number Time: 1.8648591041564941s
mosaic-features Time: 0.2920200824737549s
create-singleton-feature Time: 3.3149168491363525s
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
vote-by-classifier Time: 0.7969043254852295s
Qnodes to lookup: 7341
Qnodes from file

40it [2:05:45, 188.63s/it]


In [ ]:
def concat_files(i_path, output_file, sep=None):
    df_l = []
    for f in glob.glob(f'{i_path}/*'):
        if sep:
            df_l.append(pd.read_csv(f, sep=sep))
        else:
            df_l.append(pd.read_csv(f))
    if sep:
        pd.concat(df_l).to_csv(output_file, index=False, sep=sep)
    else:
        pd.concat(df_l).to_csv(output_file, index=False)

In [17]:
def run_context_match(features_path, context_path, custom_file, output_path):
    file_list = glob.glob(features_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        if i > -1:
            f_name = f.split('/')[-1]
            print(f'{f_name}: {i+1} of {len(file_list)}')
            context_file = f"{context_path}/{f_name[:-4]}_context.tsv"
            output_file = f"{output_path}/{f_name}"
            !tl context-match --custom-context-file $custom_file \
            --context-file $context_file --string-separator ";" \
            --similarity-string-threshold $string_threshold $f > $output_file
        

In [18]:
context_path = '/data/amandeep/nih-dataset/organization/temp/context'
features_path = '/data/amandeep/nih-dataset/organization/features'
context_output_path = '/data/amandeep/nih-dataset/organization/features_with_context'

In [19]:
run_context_match(features_path, context_path, '/data/amandeep/nih-dataset/coauthors.context.tsv.gz', context_output_path)

0it [00:00, ?it/s]

split_0_0.csv: 1 of 40


1it [08:55, 535.23s/it]

split_0_1.csv: 2 of 40


2it [17:29, 523.16s/it]

...

38it [15:31:38, 1835.00s/it]

split_3_8.csv: 39 of 40


39it [15:59:44, 1790.19s/it]

split_3_9.csv: 40 of 40


40it [16:29:43, 1484.60s/it]


In [20]:
prediction_path = '/data/amandeep/nih-dataset/organization/predictions'
!mkdir -p $prediction_path

In [21]:
features_str = ",".join(features)
def run_prediction(features_path, prediction_path):
    file_list = glob.glob(features_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{prediction_path}/{f_name}"
        !tl predict-using-model -o siamese_prediction \
        --ranking-model $ranking_model_file_path \
        --features $features_str \
        --normalization-factor $min_max_scaler_path $f > $output_file

In [22]:
run_prediction(context_output_path, prediction_path)

0it [00:00, ?it/s]

split_0_0.csv: 1 of 40
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 4.797147989273071s


1it [00:10, 10.57s/it]

split_0_1.csv: 2 of 40
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 4.323748350143433s


2it [00:20,  9.93s/it]

...

38it [07:06, 12.32s/it]

split_3_8.csv: 39 of 40
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 4.266456842422485s


39it [07:17, 12.01s/it]

split_3_9.csv: 40 of 40
/data/amandeep/github/table-linker/tl_env/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
predict-using-model Time: 4.788407325744629s


40it [07:29, 11.24s/it]


In [23]:
colorized_path = '/data/amandeep/nih-dataset/organization/colorized'
!mkdir -p $colorized_path

In [24]:
def topk_color(prediction_path, colorized_path):
    file_list = glob.glob(prediction_path + '/*.csv')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{colorized_path}/{f_name[:-4]}.xlsx"
        !tl get-kg-links -c $final_score_column -k 5 --k-rows $f \
        / add-color -c "$final_score_column" -k 5 --output "$output_file"

In [25]:
topk_color(prediction_path, colorized_path)

0it [00:00, ?it/s]

split_0_0.csv: 1 of 40
get-kg-links-siamese_prediction Time: 3.5695652961730957s
add-color Time: 0.9500653743743896s


1it [00:10, 10.02s/it]

split_0_1.csv: 2 of 40
get-kg-links-siamese_prediction Time: 3.1562962532043457s
add-color Time: 0.8754355907440186s


2it [00:19,  9.77s/it]

...

38it [06:26, 10.63s/it]

split_3_8.csv: 39 of 40
get-kg-links-siamese_prediction Time: 4.09746527671814s
add-color Time: 0.8207793235778809s


39it [06:38, 10.83s/it]

split_3_9.csv: 40 of 40
get-kg-links-siamese_prediction Time: 3.471024513244629s
add-color Time: 0.7944426536560059s


40it [06:48, 10.20s/it]


In [48]:
def add_NILS(colorized_path, output_path):
    file_list = glob.glob(colorized_path + '/*.xlsx')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{output_path}/{f_name[:-5]}.csv"
        
        df = pd.read_excel(f)
        df.loc[df['siamese_prediction'].astype(float) < 0.9, 'kg_id'] = 'NIL'
        df.to_csv(output_file, index=False)

In [26]:
def add_NIL(colorized_path, output_path):
    file_list = glob.glob(colorized_path + '/*.xlsx')
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        print(f'{f_name}: {i+1} of {len(file_list)}')
        output_file = f"{output_path}/{f_name[:-5]}.csv"
        
        df = pd.read_excel(f)
        ls = df.columns
        arr = []
        for i in range(0, len(ls)):
            arr.append('')

        arr[0] = 0
        arr[7] = 'NIL'
        arr[len(arr)-1] = ''
        df_list = df.values.tolist()
        nil = pd.DataFrame(columns = df.columns)
        new = pd.DataFrame(columns = df.columns)
        done = []
        b = []
        for i in range (0, len(df)):
            if df['row'][i] in done:
                continue


            arr[1] = df['row'][i]
            arr[2] = df['label'][i]
            arr[3] = df['context'][i]
            arr[4] = df['filename'][i]
            arr[5] = df['column-id'][i]
            arr[6] = df['label_clean'][i]


            nil.loc[len(nil)] = arr


            done.append(df['row'][i])
        nil_list = nil.values.tolist()
        for i in range(0, len(df)):
            new.loc[len(new)] = df_list[i]
            if i % 5 == 4:
                new.loc[len(new)] = (nil_list[int(i/5)])
        for i in range(0, len(new)):
            #print(new['siamese_prediction'][i])
            if (new['siamese_prediction'][i] == ''):
                new['siamese_prediction'][i] = 0
            if (new['context_score'][i] == ''):
                new['context_score'][i] = 0
            new['siamese_prediction'][i] = float(new['siamese_prediction'][i])
            new['context_score'][i] = float(new['context_score'][i])
        select = new[(new['siamese_prediction'] > siamese_threshold) | (new['kg_id'] == 'NIL')]
        select = select.reset_index(drop = True)
        final = pd.DataFrame(columns = df.columns)
        done = []
        select_list = select.values.tolist()
        for i in range(0, len(select)):
            if select['row'][i] in done:
                continue
            final.loc[len(final)] = select_list[i]
            done.append(select['row'][i])
        final.to_csv(output_file, index=False)

In [27]:
nil_path = '/data/amandeep/nih-dataset/organization/with_nils'
!mkdir -p $nil_path

In [49]:
add_NILS(colorized_path, nil_path)

0it [00:00, ?it/s]

split_0_0.xlsx: 1 of 40


1it [00:01,  1.68s/it]

split_0_1.xlsx: 2 of 40


2it [00:03,  1.56s/it]

...

38it [00:52,  1.53s/it]

split_3_8.xlsx: 39 of 40


39it [00:54,  1.47s/it]

split_3_9.xlsx: 40 of 40


40it [00:55,  1.38s/it]


In [45]:
def count_non_nils(nil_path):
    file_list = glob.glob(nil_path + '/*.csv')
    o = list()
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        o.append(pd.read_csv(f))
    df = pd.concat(o)
    print(len(df[df['column'] == 2]))
    df = df[(df['kg_id'] != 'NIL') & (df['kg_id'] != '') & (df['column'] == 2)]
    print(len(df))

        

In [46]:
count_non_nils(nil_path)

40it [00:00, 99.01it/s] 


618
618


In [50]:
def join(nil_path):
    file_list = glob.glob(nil_path + '/*.csv')
    o = list()
    for i, f in tqdm(enumerate(file_list)):
        f_name = f.split('/')[-1]
        o.append(pd.read_csv(f))
    df = pd.concat(o)
    df.to_csv('/tmp/joined.csv', index=False)

In [51]:
join(nil_path)

40it [00:02, 19.99it/s]


In [52]:
pd.read_csv(table_path, sep='\t').to_csv(table_path[:-4] + '.csv', index=False)

In [53]:
!tl join -c siamese_prediction -f /data/amandeep/nih-dataset/organization/org_for_tl_with_qnode.csv \
--extra-info /tmp/joined.csv > '/data/amandeep/nih-dataset/organization/org_for_tl_with_qnode_joined.csv'

join Time: 44.72819471359253s


In [54]:
def create_replace_nodes_mapping(joined_file, mapping_file):
    df = pd.read_csv(joined_file)
    o = []
    for i, row in df.iterrows():
        if row['name_kg_id'].strip() != "":
            o.append({
                'node1': row['org_node'],
                'label': 'same_as_item',
                'node2': row['name_kg_id'],
                'confidence': row['name_score']
            })
        if row['city_kg_id'].strip() != "":
            o.append({
                'node1': row['city_node'],
                'label': 'same_as_item',
                'node2': row['city_kg_id'],
                'confidence': row['city_score']
            })
        if row['state_kg_id'].strip() != "":
            o.append({
                'node1': row['state_node'],
                'label': 'same_as_item',
                'node2': row['state_kg_id'],
                'confidence': row['state_score']
            })
        
        if row['country_kg_id'].strip() != "":
            o.append({
                'node1': row['country_node'],
                'label': 'same_as_item',
                'node2': row['country_kg_id'],
                'confidence': row['country_score']
            })

    oo = []
    odf = pd.DataFrame(o)
    odf.drop_duplicates(subset=['node1', 'node2'], inplace=True)
    for _, gdf in odf.groupby('node1'):
        gdf = gdf[gdf['node2'] != 'NIL']
        if len(gdf) > 1:
            oo.append(pd.DataFrame(gdf.head(0)))
        else:
            oo.append(gdf)
    pd.concat(oo).to_csv(mapping_file, sep='\t', index=False)

In [55]:
create_replace_nodes_mapping('/data/amandeep/nih-dataset/organization/org_for_tl_with_qnode_joined.csv', '/data/amandeep/nih-dataset/organization/replace_nodes_mapping.tsv')

In [60]:
def concat_org_files():
    f_p = "/data/amandeep/nih-dataset/organization/kgtk-files-nih-V3.0"
    o = []
    for f in glob.glob(f"{f_p}/*tsv"):
        o.append(pd.read_csv(f, sep='\t'))
    df = pd.concat(o)
    df.to_csv('/data/amandeep/nih-dataset/organization/nih-org-kgtk.tsv', sep='\t', index=False)

In [61]:
concat_org_files()

kgtk replace-nodes -i nih-org-kgtk.tsv -o nih-org-kgtk-wikidata-qnodes.tsv --mapping-file replace_nodes_mapping.tsv